In [34]:
import pandas as pd
import numpy as np
import random

In [35]:
data = pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/data/modeling data/modeling_data.csv', encoding = 'CP949')

In [36]:
random.seed(2020)

In [37]:
# feature, target 설정
df_num = data

In [38]:
df_num

,Unnamed: 0,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,1,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
1,2,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,3,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,4,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,0,11,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,5,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,1,0,7.066787,True,1.562276e+13,42767.077,10.591,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057389,1057390,충북,휴양콘도 운영업,충북,70s,2,5,230400.0,1,2020,0,0,7.066787,False,4.267602e+11,42767.080,10.591,NaN
1057390,1057391,충북,휴양콘도 운영업,충북,70s,2,5,247800.0,12,2019,0,0,8.806154,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057391,1057392,충북,휴양콘도 운영업,충북,70s,2,5,83000.0,2,2019,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057392,1057393,충북,휴양콘도 운영업,충북,70s,2,5,329800.0,5,2019,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05


In [39]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['AMT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [40]:
# 예측 템플릿 만들기 : total indst 기준으로 돌리자.
temp = pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/data/modeling data/template.csv', encoding = 'CP949')

In [41]:
temp = temp.drop(['home_ratio'],axis = 1)

In [42]:
#log_temp = pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/data/modeling data/log_template.csv', encoding = 'CP949')

In [43]:
type(temp)

pandas.core.frame.DataFrame

In [44]:
temp.shape

(1658860, 14)

In [45]:
temp.head(3)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,rain_day,age_population,total_sale_by_indst_groupby,total_tourist_groupby,covid,festival_count
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,8.806154,15622761472137,42767.077,NaN,True,0
1,강원,건강보조식품 소매업,강원,20s,1,1,7,2020,11.461487,15622761472137,42767.077,NaN,True,0
2,강원,건강보조식품 소매업,강원,20s,1,2,4,2020,8.806154,15622761472137,42767.077,NaN,True,0


In [46]:
#lightGBM
x = train_num.drop(['AMT','Unnamed: 0','log_total_sale_by_indst','deceased'], axis=1) # total indst 기준으로 돌리자.
y = np.log1p(train_num['AMT'])

In [47]:
x.dtypes

CARD_SIDO_NM                    object
STD_CLSS_NM                     object
HOM_SIDO_NM                     object
AGE                             object
SEX_CTGO_CD                      int64
FLC                              int64
month                            int64
year                             int64
festival_count                   int64
rain_day                       float64
covid                             bool
age_population                 float64
total_sale_by_indst_groupby    float64
total_tourist_groupby          float64
dtype: object

In [48]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid']
for i in enumerate (cat_features) : 
    ca = i[1] 
    x[ca] = x[ca].astype('category') 

In [49]:
k = int(len(x)*0.9)

In [50]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [51]:
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

In [52]:
import lightgbm as lgb

In [53]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [54]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [55]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.75004
[200]	valid_0's rmse: 1.46805
[300]	valid_0's rmse: 1.30613
[400]	valid_0's rmse: 1.19732
[500]	valid_0's rmse: 1.12167
[600]	valid_0's rmse: 1.06165
[700]	valid_0's rmse: 1.00728
[800]	valid_0's rmse: 0.968559
[900]	valid_0's rmse: 0.937952
[1000]	valid_0's rmse: 0.908782
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.908782


-------------------------------문제시작---------------------------------

In [56]:
x_train.dtypes==x_val.dtypes

CARD_SIDO_NM                   True
STD_CLSS_NM                    True
HOM_SIDO_NM                    True
AGE                            True
SEX_CTGO_CD                    True
FLC                            True
month                          True
year                           True
festival_count                 True
rain_day                       True
covid                          True
age_population                 True
total_sale_by_indst_groupby    True
total_tourist_groupby          True
dtype: bool

In [57]:
x_train_col = x_train.columns.to_list()
temp_col = temp.columns.to_list()

In [58]:
list(set(temp_col) - set(x_train_col))

[]

In [59]:
obj_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE','covid']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp[ca] = temp[ca].astype('category') 

obj_features = ['year', 'month']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp[ca] = temp[ca].astype('int64') 

In [60]:
temp.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                       int64
FLC                               int64
month                             int64
year                              int64
rain_day                        float64
age_population                    int64
total_sale_by_indst_groupby     float64
total_tourist_groupby           float64
covid                          category
festival_count                    int64
dtype: object

In [61]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100+ temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [62]:
#디코딩?
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM']
for i in enumerate (cat_features) : 
    ca = i[1] 
    temp[ca] = temp[ca].astype('object') 

In [63]:
temp.head(3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,112064136.0
1,202004,강원,골프장 운영업,312940460.0
2,202004,강원,과실 및 채소 소매업,282214691.0


In [65]:
# 제출 파일 만들기
submission = pd.read_csv('C:/Users/USER/Desktop/[2020]/2020 summer/데이콘/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,112064136.0
1,202004,강원,골프장 운영업,312940460.0
2,202004,강원,과실 및 채소 소매업,282214691.0
3,202004,강원,관광 민예품 및 선물용품 소매업,29496603.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,11910703.0


In [66]:
submission.to_csv('C:/Users/USER/Desktop/[2020]/submission.csv', encoding='utf-8-sig')